In [4]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

path='data/APPENDED'
data_path='data'
ref_dataset= np.load(os.path.join(path,'refrigerator.dat.npy'))
print(ref_dataset[0])
ref_bins=4
ref_window_size=2401
input_shape=(2401, 1)

ref_dataset=ref_dataset[:,1:]
print(ref_dataset.shape)

bin_max_count=[0 for i in range(ref_bins)]
ref_clusters=[6.64953227,193.52627165,456.50688195]

[1.30313293e+09 2.22200000e+02 6.00000000e+00]
(1575991, 2)


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization
from keras import regularizers

def load_cnn():
    
    model=Sequential()

    model.add(Conv1D(filters=8,kernel_size=5,input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    #model.add(Dense(100,activation='relu'))
    model.add(Dropout(0.5))
#    model.add(Dense(256,activation='relu'))
#    model.add(Dense(128,activation='relu'))
#    model.add(Dense(64,activation='relu'))
#    model.add(Dense(32,activation='relu'))
    model.add(Dense(3,activation='sigmoid'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 1e-6
    momentum = 0.8
    opt= Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [6]:
import collections
from keras.utils import np_utils
from matplotlib import pyplot as plt
from keras.utils import normalize
import csv


bin_dict=collections.Counter(ref_dataset[:,1])

ref_data=[]
ref_target=[]
appliance_original=[]

model=load_cnn()
k=0
count=0
graph_no=0
key_min = min(bin_dict.keys(), key=(lambda k: bin_dict[k]))
ref_min_bin=bin_dict[key_min]
print(ref_min_bin)

epo=0
while(epo<20):
    graph_no=0
    for i in range(ref_dataset.shape[0]-ref_window_size-1-200000):
        ref_data.append(ref_dataset[i:i+ref_window_size,0])

        appliance=ref_dataset[i+ref_window_size-1,1]
        appliance_original.append(appliance)
        
        total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
        prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
        prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
        prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
        #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
        #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))
        ref_target.append([prob0,prob1,prob2])
        k=k+1
        if(k==10000):
            data=np.array(ref_data)/500.0
            data = normalize(data, axis=1)
            target=np.array(ref_target)
            data=data.reshape(data.shape[0],data.shape[1],1)
            #print(target.shape,data[0],target[0])

            if(count!=10):
                history=model.fit(data,target,epochs=2) 

                ref_data=[]
                ref_target=[]
                appliance_original=[]
                k=0
                count=count+1

            else:
    #            ref_data=[]
    #            ref_target=[]        
    #            for i in range(10000):
    #                ref_data.append(ref_dataset[700000+i:700000+i+ref_window_size,0])

    #                appliance=ref_dataset[700000+i+ref_window_size-1,1]
                    #print(appliance)

    #                total=np.sum(np.array([1/np.power(ref_clusters[j]-appliance,2) for j in range (len(ref_clusters))]))
    #                prob0=(1/np.power(ref_clusters[0]-appliance,2))/total
    #                prob1=(1/np.power(ref_clusters[1]-appliance,2))/total
    #                prob2=(1/np.power(ref_clusters[2]-appliance,2))/total
                    #prob3=(1/np.power(ref_clusters[3]-appliance[i],2))/total    
                    #print(appliance,[prob0,prob1,prob2],np.sum([prob0*[ref_clusters[0]],prob1*[ref_clusters[1]],prob2*[ref_clusters[2]]]))

    #                ref_target.append([prob0,prob1,prob2])

    #            data=np.array(ref_data)/500.0
    #            data = normalize(data, axis=1)
    #            target=np.array(ref_target)
    #            data=data.reshape(data.shape[0],data.shape[1],1)

                print(model.evaluate(data,target),epo,graph_no)
                results=model.predict(data)
                #print(results.shape,results,results[0].shape,results[0])
                results_ori=[bins[0]*ref_clusters[0]+bins[1]*ref_clusters[1]+bins[2]*ref_clusters[2] for bins in results]
                #print(appliance_original)
                #print(results_ori)
                
                plt.plot(results_ori,'r--',label='PREDICTED')
                plt.plot(appliance_original,'b',label='ACTUAL')
                plt.legend()
                plt.savefig('graphs_soft_association/'+str(epo)+str(graph_no)+'.jpg')
                plt.close()

                ref_data=[]
                ref_target=[]
                appliance_original=[]
                k=0
                count=0
                graph_no=graph_no+1
    epo=epo+1

3
Epoch 1/2
10000/10000 [==============================] - 31s 3ms/step - loss: 0.2586 - acc: 0.9037
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0770 - acc: 0.9840
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.1231 - acc: 0.9680
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0273 - acc: 0.9950
Epoch 1/2
10000/10000 [==============================] - 31s 3ms/step - loss: 0.0852 - acc: 0.9761
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0431 - acc: 0.9863
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0611 - acc: 0.9842
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0257 - acc: 0.9949
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0732 - acc: 0.9744
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0221 - acc: 0.9955
Epoch 1/

10000/10000 [==============================] - 28s 3ms/step - loss: 0.0664 - acc: 0.9789 - ETA: 1s - loss
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1314 - acc: 0.9535
Epoch 2/2
10000/10000 [==============================] - 10s 961us/step
[4.718039317186154, 0.6445] 0 3
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0470 - acc: 0.9907
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0128 - acc: 0.9974
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0797 - acc: 0.9723
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0238 - acc: 0.9940
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0286 - acc: 0.9917
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0165 - acc: 0.9958
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0112 - acc: 0.997

10000/10000 [==============================] - 28s 3ms/step - loss: 0.0174 - acc: 0.9951
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0145 - acc: 0.9958
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0864 - acc: 0.9773: 0s - loss: 0.0883 - ac
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0307 - acc: 0.9945
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1479 - acc: 0.9542
Epoch 2/2
10000/10000 [==============================] - 10s 966us/step
[2.042009750217851, 0.7918] 0 7
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1596 - acc: 0.9547
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0449 - acc: 0.9880
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0802 - acc: 0.9723
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0438 - acc

10000/10000 [==============================] - 28s 3ms/step - loss: 0.5820 - acc: 0.7934
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.2243 - acc: 0.8931
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0722 - acc: 0.9809: 0s - loss: 0.0724 - acc: 0.
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0151 - acc: 0.9978
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0194 - acc: 0.9961
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0129 - acc: 0.9983
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0162 - acc: 0.9966
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0138 - acc: 0.9976
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0967 - acc: 0.9720
Epoch 2/2
10000/10000 [==============================] - 9s 950us/step
[0.1665007354163797

10000/10000 [==============================] - 28s 3ms/step - loss: 0.0199 - acc: 0.9932
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0912 - acc: 0.9733
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0372 - acc: 0.9912: 2s - lo
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0649 - acc: 0.9772
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0362 - acc: 0.9887
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0424 - acc: 0.9853
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0283 - acc: 0.9890
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0352 - acc: 0.9846
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0270 - acc: 0.9888
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0280 - acc: 0.9917
Epoch 2/2
1

...

10000/10000 [==============================] - 29s 3ms/step - loss: 0.0924 - acc: 0.9761
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.5265 - acc: 0.8368
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.3087 - acc: 0.8960
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.1611 - acc: 0.9373
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0716 - acc: 0.9729
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0478 - acc: 0.9861
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0388 - acc: 0.9886:
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0707 - acc: 0.9806
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0345 - acc: 0.9903
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0989 - acc: 0.9623
Epoch 2/2
10000/100

10000/10000 [==============================] - 10s 996us/step
[0.08894820003637578, 0.967] 18 10
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0518 - acc: 0.9816
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0369 - acc: 0.9870
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0420 - acc: 0.9864
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0389 - acc: 0.9871
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0501 - acc: 0.9833
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0399 - acc: 0.9872
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0494 - acc: 0.9842
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0437 - acc: 0.9857
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.3197 - acc: 0.9075
Epoch 2/2
10

10000/10000 [==============================] - 28s 3ms/step - loss: 0.1606 - acc: 0.9451
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0857 - acc: 0.9717
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0370 - acc: 0.9906
Epoch 2/2
10000/10000 [==============================] - 10s 1ms/step
[0.33761520832681563, 0.8268] 19 1
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1679 - acc: 0.9457
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1091 - acc: 0.9625
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0860 - acc: 0.9742
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0580 - acc: 0.9849
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0406 - acc: 0.9895: 
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0352 - acc: 0.9912
Epoch 1/2
10

10000/10000 [==============================] - 28s 3ms/step - loss: 0.0539 - acc: 0.9808
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.1026 - acc: 0.9666: 0s - loss: 0.1023 - acc: 0.9
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0930 - acc: 0.9711
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0385 - acc: 0.9855
Epoch 2/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0252 - acc: 0.9916
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.1017 - acc: 0.9684
Epoch 2/2
10000/10000 [==============================] - 10s 996us/step
[0.26793136467649603, 0.8975] 19 5
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0654 - acc: 0.9785
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0492 - acc: 0.9849
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0

10000/10000 [==============================] - 29s 3ms/step - loss: 0.0261 - acc: 0.9923
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0932 - acc: 0.9675
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0581 - acc: 0.9783
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.1312 - acc: 0.9527
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0800 - acc: 0.9714
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0885 - acc: 0.9639
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0519 - acc: 0.9819
Epoch 1/2
10000/10000 [==============================] - 28s 3ms/step - loss: 0.0403 - acc: 0.9858
Epoch 2/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0363 - acc: 0.9884
Epoch 1/2
10000/10000 [==============================] - 29s 3ms/step - loss: 0.0632 - acc: 0.9785
Epoch 2/2
10000/1000

In [ ]:
np.argmax(test_target,axis=1).shape

In [ ]:
test_target.shape

In [ ]:
test_target

In [ ]:
np.argmax(test_target,axis=1)

In [ ]:
model.summary()